# Convert multiple games to dataframe
Will run through multiple games to produce a dataframe of common kill locations

In [1]:
# find files
from os import listdir

mypath = "C:\\Users\kevin\Documents\BTS\KT_Work\Heroes of the Storm\\replays\Heroes lounge\season 16 division 3\goofy looking guys"

replay_files = listdir(mypath)

In [2]:
# update heroprotocol with terminal
!python -m pip install --upgrade heroprotocol --user --no-warn-script-location

Requirement already up-to-date: heroprotocol in c:\users\kevin\appdata\roaming\python\python37\site-packages (2.55.0.86938)


In [2]:
# import function
from replay_to_dataframe import *
# import replay_to_dataframe as r2d

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
replay_i = 1
for replay_file in replay_files:
# replay_file = replay_files[1]
    !python -m heroprotocol --gameevents {'"'+mypath+"\\"+replay_file+'"'} > gameevents.txt
    !python -m heroprotocol --messageevents {'"'+mypath+"\\"+replay_file+'"'} > messageevents.txt
    !python -m heroprotocol --trackerevents {'"'+mypath+"\\"+replay_file+'"'} > trackerevents.txt
    !python -m heroprotocol --attributeevents {'"'+mypath+"\\"+replay_file+'"'} > attributeevents.txt
    !python -m heroprotocol --header {'"'+mypath+"\\"+replay_file+'"'} > header.txt
    !python -m heroprotocol --details {'"'+mypath+"\\"+replay_file+'"'} > details.txt
    !python -m heroprotocol --initdata {'"'+mypath+"\\"+replay_file+'"'} > initdata.txt

    # parse files
    data = ReplayData(near_distance = 10)

    # set up export df
    export_df = data.heroes_died_position.copy()

    # add level of each team
    exp_df = data.exp_event[["gameloop", "team", "level"]].copy()
    exp_df["team"] = exp_df["team"] - 1

    export_df.at[:,"team0_lvl"] = 1
    export_df.at[:,"team1_lvl"] = 1

    export_df.at[:,"all_players"] = ""

    for row_i in export_df.index:
        gl = closest_gameloop(export_df.iloc[row_i].died_gameloop, list(map(int, exp_df.gameloop)))

        if gl == -1:
            pass
        else:
            export_df.at[row_i, 
                         ["team0_lvl", "team1_lvl", "all_players"]
                        ] = [exp_df.loc[(exp_df['gameloop'] == gl) &
                                        (exp_df['team'] == 0)]['level'].item(),
                             exp_df.loc[(exp_df['gameloop'] == gl) &
                                        (exp_df['team'] == 1)]['level'].item(),
                             list(data.player_info.m_name)]

        # add playerlist


    # add map
    export_df.at[:,"map"] = data.map

    # add time played
    export_df.at[:,"timeUTC"] = data.timeUTC

    # add filename
    export_df.at[:,"replay_file"] = replay_file.split(".")[0]



    # export as csv
    export_df.to_csv("./temp/" + replay_file.split(".")[0] + ".csv")

    # building location
    #     data.export_to_csv("building_position","./temp/" + data.map + "_" + replay_file.split(".")[0] + ".csv")
    data.building_position.to_csv("./building_position/" + data.map + "_" + replay_file.split(".")[0] + ".csv")

    print("(" + str(replay_i) + " of " + str(len(replay_files)) + ") " + replay_file + " parsed")
    replay_i += 1

(0 of 24) 613e664d55853496476135.stormreplay parsed


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


(1 of 24) 613e665d074fd281805927.stormreplay parsed


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


(2 of 24) 613e66683955a856996939.stormreplay parsed
(3 of 24) 614b8fed79580961488363.stormreplay parsed
(4 of 24) 614b900117b07807174774.stormreplay parsed
(5 of 24) 614b9019c8891299171987.stormreplay parsed
(6 of 24) 6154ba4451a25185507045.stormreplay parsed
(7 of 24) 6154ba5774c20194842551.stormreplay parsed
(8 of 24) 615b611303473578598884.stormreplay parsed
(9 of 24) 615b611d77166008120536.stormreplay parsed
(10 of 24) 616882a35e971229109478.stormreplay parsed
(11 of 24) 616882b9256e7260067818.stormreplay parsed
(12 of 24) 6170751aa7b0d813705876.stormreplay parsed
(13 of 24) 617075226991d523291605.stormreplay parsed
(14 of 24) 617fe9a9c8205763594624.stormreplay parsed
(15 of 24) 617fe9bc45ee0453184808.stormreplay parsed
(16 of 24) 61883d002f552195323257.stormreplay parsed
(17 of 24) 61883d112e4ac706910189.stormreplay parsed
(18 of 24) 61883d22afbd4862592431.stormreplay parsed
(19 of 24) 618ae54d1f4f7333916240.stormreplay parsed
(20 of 24) 618ae55a31767810887320.stormreplay parsed
(

In [3]:
# combine all CSVs in a dataframe
mypath_temp = "C:\\Users\kevin\Documents\BTS\KT_Work\Heroes of the Storm\\temp"
temp_replay_prased_files = listdir(mypath_temp)

li = []

for filename in temp_replay_prased_files:
    df = pd.read_csv(mypath_temp+"\\"+filename, header=0)
    li.append(df)

hero_died_all = pd.concat(li, axis=0, ignore_index=True)

In [4]:
# translate map names to English
print(set(hero_died_all.map))

map_name_translation = {
    'Drachengärten': 'Dragon Shire',
    'Grabkammer der Spinnenkönigin': 'Tomb of the Spider Queen', 
    'Höllenschreine': 'Infernal Shrines'
}

hero_died_all.loc[:,"map"] = hero_died_all.map.replace(map_name_translation)

print(set(hero_died_all.map))

{'Dragon Shire', 'Towers of Doom', 'Garden of Terror', 'Drachengärten', 'Grabkammer der Spinnenkönigin', 'Cursed Hollow', 'Alterac Pass', 'Höllenschreine', 'Sky Temple', 'Infernal Shrines', 'Volskaya Foundry'}
{'Dragon Shire', 'Tomb of the Spider Queen', 'Towers of Doom', 'Garden of Terror', 'Cursed Hollow', 'Alterac Pass', 'Sky Temple', 'Infernal Shrines', 'Volskaya Foundry'}


In [7]:
# translate map names for building position csv
import os

mypath_building_position = "C:\\Users\kevin\Documents\BTS\KT_Work\Heroes of the Storm\\building_position"
building_position_files = listdir(mypath_building_position)

for filename in building_position_files:
    map_name = filename.split("_")[0]
    replay_name = filename.split("_")[1]
    if map_name in map_name_translation.keys():
        old_file_name = mypath_building_position + "\\" + filename
        new_file_name = mypath_building_position + "\\" + map_name_translation[map_name] + "_" + replay_name
        
        os.rename(old_file_name, new_file_name)
            

In [8]:
# combine building positions
building_position_files = listdir(mypath_building_position)

li = []

for filename in building_position_files:
    df = pd.read_csv(mypath_building_position+"\\"+filename, header=0)
    # add map name
    df.at[:,"map_name"] = filename.split("_")[0]
    li.append(df)

building_position_all = pd.concat(li, axis=0, ignore_index=True)

The building position do not vary between replays but change on different maps

In [20]:
# fill all_players with cell below
hero_died_all.loc[:,"all_players"] = hero_died_all.all_players.bfill()

In [21]:
# output 
hero_died_all.to_csv("hero_died_all.csv")
building_position_all.to_csv("building_position_all.csv")